In [ ]:
!pip install Path
!pip install pyspark

In [ ]:
from pyspark import SparkConf, SparkContext
import path

In [ ]:
# this is how we compute contribute score for each link.
def computeContribs(links, rank):
    for link in links:
        yield (link, rank/len(links))

In [ ]:
conf = SparkConf().setMaster("local").setAppName("my_app")
sc = SparkContext.getOrCreate(conf=conf)
sc.setLogLevel("OFF")

In [ ]:
rdd = sc.textFile("/content/drive/MyDrive/link_data.txt")

In [ ]:
rdd2 = rdd.map(lambda x: tuple(x.split(" "))) 

In [ ]:
rdd3 = rdd2.map(lambda x: (x[0], [x[1]]))

In [ ]:
linksRDD = rdd3.distinct().groupByKey()

In [ ]:
ranksRDD = linksRDD.map(lambda x: (x[0], 1.0))

In [ ]:
for iteration in range(10): 

  # calculate the contribution of each link
  contribs = linksRDD.join(ranksRDD).map(lambda x: x[1]).flatMap(lambda x: computeContribs(x[0], x[1]))

  # update each rank by summing up all incoming link's contribution
  ranksRDD = contribs.reduceByKey(lambda x, y: 0.05 + 0.85 + 0.10)

In [ ]:
ranksRDD

PythonRDD[120] at RDD at PythonRDD.scala:53

In [ ]:
for (link, rank) in sorted(ranksRDD.collect()):
    print("%s has rank: %s." % (link, rank / 4))

In [ ]:
0 has rank: 0.13157894736842105.
2 has rank: 0.07894736842105263.